In [1]:
import pandas as pd
import io

In [2]:
pwd

'C:\\Users\\rozy1'

In [3]:

df = pd.read_csv(r"C:\\Users\\rozy1\\myFile\\trainDataKagelSW.csv")


df['class'].value_counts()

0    32245
1    23650
Name: class, dtype: int64

In [4]:

dft =  pd.read_csv(r"C:\\Users\\rozy1\\myFile\\testDataOwn.csv")

dft['class'].value_counts()

0    4561
1    1397
Name: class, dtype: int64

In [5]:
df_spam = df[df['class']== 1]
df_spam.shape

df_ham = df[df['class']== 0]
df_ham.shape

df_ham_down = df_ham.sample(df_spam.shape[0], replace=True)
df_ham_down.shape

df_balanced = pd.concat([df_spam, df_ham_down])
df_balanced.shape

df_balanced['class'].value_counts()

1    23650
0    23650
Name: class, dtype: int64

In [6]:
dft_spam = dft[dft['class']== 1]
dft_spam.shape

dft_ham = dft[dft['class']== 0]
dft_ham.shape

dft_ham_down = dft_ham.sample(dft_spam.shape[0], replace=True)
dft_ham_down.shape

dft_balanced = pd.concat([dft_spam, dft_ham_down])
dft_balanced.shape

dft_balanced['class'].value_counts()

1    1397
0    1397
Name: class, dtype: int64

In [7]:
import tensorflow_hub as hub
import tensorflow_text as text

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics.pairwise import cosine_similarity

# Split data into X and y
X_train = df_balanced['sentence']
y_train = df_balanced['class']
X_test = dft_balanced['sentence']
y_test = dft_balanced['class']

In [11]:
# Clear the TensorFlow Hub cache
hub.cache.clear()

AttributeError: module 'tensorflow_hub' has no attribute 'cache'

In [15]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
endoded_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(endoded_url)

ValueError: Trying to load a model of incompatible/unknown type. 'C:\Users\rozy1\AppData\Local\Temp\tfhub_modules\602d30248ff7929470db09f7385fc895e9ceb4c0' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.

In [ ]:
def get_sentence_embeding(sen):
  preprocess_text = bert_preprocess(sen)
  return bert_encoder(preprocess_text)['pooled_output']

In [ ]:
# bert layers
text_input = tf.keras.layers.Input(shape=(), dtype = tf.string, name ="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# neural network layers 

l = tf.keras.layers.Dropout(0.1, name='dropout1')(outputs['pooled_output']) 

l = tf.keras.layers.Dense(512, activation='relu', name='dense1')(l) 

l = tf.keras.layers.BatchNormalization(name='batch_norm1')(l) 

l = tf.keras.layers.Dropout(0.2, name='dropout2')(l) 

l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l) 

# final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [ ]:
model.summary()

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping

METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),

    tf.keras.metrics.MeanSquaredError(name='mse'),
    tf.keras.metrics.RootMeanSquaredError(name='rmse')
]


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

# Add early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[early_stop])

In [ ]:
from sklearn.metrics import f1_score

# Evaluate the model on the test set
results = model.evaluate(X_test, y_test)

# Extract the model's predictions on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype('int32')  # Convert probabilities to binary predictions

# Calculate the F1 score
f1 = f1_score(y_test, y_pred)

# Print the F1 score
print('Test set F1 score:', f1)


In [ ]:
y_predicted = y_pred.flatten()

In [ ]:
import seaborn as sn
from matplotlib import pyplot as plt

sn.heatmap(confusion_matrix(y_test, (y_predicted > 0.5)), annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, (y_predicted > 0.5)))

In [ ]:
import time

start_time = time.time()

reviews=['youu got wild bitches telling you lies']
prediction = model.predict(reviews)

end_time = time.time()

print("Prediction time:", end_time - start_time, "seconds")
print("Prediction:", prediction)